# Bonus certificate calculation

In [1]:
from FFTpricing import fft_call_price_bs
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

In [14]:
fft_call_price_bs(1000, 0.04, 0.2, 1, eta=0.01, N=2**20, alpha=1.5,  strike=[100,150,200, 300, 1000, 3000, 10000])

array([ 9.03921056e+02,  8.55881584e+02,  8.07842112e+02,  7.11763168e+02,
        9.92505372e+01,  3.60506924e-06, -7.97236623e-16])

In [ ]:
N=4096*16
eta =0.25

lambd = 2 * np.pi / (N * eta)
b = 0.5 * N * lambd
k = -b + lambd * np.arange(N)  # log-strike grid
K = np.exp(k) 

In [16]:
K=10000
T_BS = 1
vol_basket = 0.2
ci = 0.04
dividend_effective_portfolio = 0
# basket price
S_basket = 1000

# black-scholes
d_1 = (np.log(S_basket/K)+(ci-dividend_effective_portfolio+vol_basket**2/2)*T_BS)/(vol_basket*np.sqrt(T_BS))
d_2 = d_1 - (vol_basket*np.sqrt(T_BS))

call_option = S_basket*np.exp(-dividend_effective_portfolio*T_BS)*norm.cdf(d_1)-K*np.exp(-ci*T_BS)*norm.cdf(d_2)

# price of the call option
print(call_option)

3.0412377173844148e-28
